![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## This notebook is an example of how to pipeline two models. 
This notebook is an example of how to use DeGirum PySDK to do AI inference of a graphical file using 
two AI models: face detection and gender classification. The face detection model 
is run on the image and the results are then processed by the gender classification model, 
one face at a time. Combined result is then displayed.
This script uses PIL as image processing backend.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option. 

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where do you want to run your inferences

In [ ]:
# hw_location: where you want to run inference.
#     Use "@cloud" to use DeGirum cloud.
#     Use "@local" to run on local machine.
#     Use an IP address for AI server inference.#
# face_model_zoo_url: URL/path for the face model zoo.
#     Use cloud_zoo_url for @cloud, @local, and AI server inference options.
#     Use '' for an AI server serving models from a local folder.
#     Use a path to a JSON file for a single model zoo in case of @local inference.#
# face_model_name: name of the model for face detection.
# gender_model_zoo_url: URL/path for the gender model zoo.
# gender_model_name: name of the model for gender detection.
# video_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
hw_location = "@cloud"
face_model_zoo_url = "https://cs.degirum.com/degirum/ultralytics_v6"
face_model_name = "yolov8n_relu6_face--640x640_quant_n2x_orca1_1"
age_gender_model_zoo_url = "https://cs.degirum.com/degirum/AgeGenderCls"
gender_model_name = "mobilenetv2_050_gender--160x160_quant_n2x_orca1_1"
age_model_name = "yolov8n_relu6_age--256x256_quant_n2x_orca1_3"
video_source = (
    "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/faces_and_gender.mp4"
)              

#### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg
import degirum_tools

# Connect to AI inference engine 
face_zoo = dg.connect(hw_location, face_model_zoo_url, degirum_tools.get_token())
age_gender_zoo = dg.connect(hw_location, age_gender_model_zoo_url, degirum_tools.get_token())

In [ ]:
# Load face and gender models
with face_zoo.load_model(face_model_name) as face_model:
    with age_gender_zoo.load_model(gender_model_name) as gender_model:
        # Create a compound cropping model with 50% crop extent
        crop_model = degirum_tools.CroppingAndClassifyingCompoundModel(
            face_model, gender_model, 50.0
        )

        # Detect faces and genders
        with degirum_tools.Display("Faces and Gender") as display:
            for gender_results in degirum_tools.predict_stream(crop_model, video_source):
                display.show(gender_results)

In [ ]:
# Load face and age models
with face_zoo.load_model(face_model_name) as face_model:
    with age_gender_zoo.load_model(age_model_name, overlay_show_probabilities=True) as age_model:
        # Create a compound cropping model with 50% crop extent
        crop_model = degirum_tools.CroppingAndClassifyingCompoundModel(
            face_model, age_model, 50.0
        )

        # Detect faces and genders
        with degirum_tools.Display("Faces and Age") as display:
            for age_results in degirum_tools.predict_stream(crop_model, video_source):
                display.show(age_results)